# Biomethane-to-Ammonia - LCIA results

This notebook performs the life cycle impact assessment (LCIA) calculations (and some extra calculations) and export the results

In [2]:
%run _imports.ipynb

In [3]:
# Import some external data used in the calculations
biomethane_potential = pd.read_excel(DATA_DIR / "sustainable_biomethane_potential_europe.xlsx", sheet_name="Biomethane Potential", index_col=[0])
ammonia_production = pd.read_excel(DATA_DIR / "ammonia_production_europe.xlsx", sheet_name='Data')

## Carbon footprint of ammonia production

### Europe average carbon footprint & impact breakdown

In [4]:
carbon_footprint_ammonia_RER = {}

for inv in INVENTORIES:
    activity = [a for a in bw.Database(LCI_DB) if a['name'] == INVENTORIES[inv][0] 
                                               and a['reference product'] == INVENTORIES[inv][1]
                                               and a['location'] == 'RER'][0]
    lca_results = results_analysis.lcia_system_contribution(activity, IPCC_METHOD, activity_amount=1)
    for impact in lca_results:
        if impact not in carbon_footprint_ammonia_RER:
            carbon_footprint_ammonia_RER[impact] = {}
        carbon_footprint_ammonia_RER[impact].update({inv: lca_results[impact]})

carbon_footprint_ammonia_RER_df = pd.DataFrame(carbon_footprint_ammonia_RER['Climate change, GWP 100a'])

In [ ]:
# Save results to csv
#carbon_footprint_ammonia_RER_df.to_csv(DATA_DIR / "results" / f"Fig 2 Carbon footprint ammonia production Europe average_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv")

**Carbon footprint with emissions from ammonia usage as fertilizer or shipping fuel**

In [65]:
# N2O emission factors for mineral fertilizers based on IPCC Chapter 11
# 1.6% of nitrogen in mineral fertilizer for wet climates
# 0.5% of nitrogen in mineral fertilizer for dry climates
# 0.4% of nitrogen in ammonia fuel in ships
ammonia_nitrogen_content = 0.82 # anyhdrous ammonia is 82% nitrogen
cf_n2o = 273.0

n2o_ef_wet = 0.016
n2o_ef_dry = 0.005
n2o_emissions_wet = 1 * ammonia_nitrogen_content * n2o_ef_wet * 44/28
n2o_emissions_dry = 1 * ammonia_nitrogen_content * n2o_ef_dry * 44/28
fertilizer_use_impact_wet = n2o_emissions_wet * cf_n2o
fertilizer_use_impact_dry = n2o_emissions_dry * cf_n2o

n2o_fuel = 0.004
n2o_emissions_fuel = 1 * ammonia_nitrogen_content * n2o_fuel * 44/28
fuel_use_impact = n2o_emissions_fuel * cf_n2o

carbon_footprint_with_use = {}
carbon_footprint_with_use["Fertilizer low"] = {}
carbon_footprint_with_use["Fertilizer high"] = {}
carbon_footprint_with_use["Fuel"] = {}

for scenario in carbon_footprint_ammonia_RER_df.columns:
    carbon_footprint_with_use["Fertilizer low"][scenario] = carbon_footprint_ammonia_RER_df.loc["Total"][scenario] + fertilizer_use_impact_dry
    carbon_footprint_with_use["Fertilizer high"][scenario] = carbon_footprint_ammonia_RER_df.loc["Total"][scenario] + fertilizer_use_impact_wet
    carbon_footprint_with_use["Fuel"][scenario] = carbon_footprint_ammonia_RER_df.loc["Total"][scenario] + fuel_use_impact

carbon_footprint_with_use = pd.DataFrame(carbon_footprint_with_use)

In [66]:
# Save results to csv
#carbon_footprint_with_use.to_csv(DATA_DIR / "results" / f"Fig S2 Carbon footprint ammonia production Europe average with usage_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv")

### Country-specific carbon footprint / total impact

In [5]:
# List of countries
LIST_COUNTRIES = [c for c in biomethane_potential.T.index if c not in ['Unit', 'Feedstock', 'Total']]
LIST_COUNTRIES[:5]

['DE', 'FR', 'IT', 'ES', 'PL']

In [6]:
carbon_footprint_ammonia_country = {}

for inv in {k: v for (k,v) in INVENTORIES.items() if k != 'Green H2'}:
    carbon_footprint_ammonia_country[inv] = {}
    for country in LIST_COUNTRIES:
        activity = [a for a in bw.Database(LCI_DB) if a['name'] == INVENTORIES[inv][0] 
                                                      and a['reference product'] == INVENTORIES[inv][1]
                                                      and a['location'] == country][0]
        impacts_loc = results_analysis.multi_lcia(activity, IPCC_METHOD, amount=1)
        carbon_footprint_ammonia_country[inv].update({country: impacts_loc['Climate change, GWP 100a']})

In [7]:
carbon_footprint_ammonia_country_df = pd.DataFrame(carbon_footprint_ammonia_country)

ammonia_country_only_biomethane = carbon_footprint_ammonia_country_df[['Biomethane',
                                                                        'Biomethane + CCS Syngas',
                                                                        'Biomethane + CCS Syngas + CCS Heating',
                                                                        'Biomethane + CCS Upgrading',
                                                                        'Biomethane + CCS Upgrading + CCS Syngas', 
                                                                        'Biomethane + CCS Upgrading + CCS Syngas + CCS Heating']]

In [ ]:
# Save results to csv
#ammonia_country_only_biomethane.to_csv(DATA_DIR / "results" / f"Fig 3 Country-specific carbon footprint ammonia production_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv")

## Biomethane and ammonia production potentials in Europe

In [8]:
# Biomethane potential by country (bcm/year)
biomethane_ammonia_potentials = biomethane_potential.loc['Sustainable biomethane potential in 2030']
biomethane_ammonia_potentials = biomethane_ammonia_potentials[biomethane_ammonia_potentials['Feedstock'] == 'TOTAL']
biomethane_ammonia_potentials.drop(['Unit', 'Feedstock'], axis=1, inplace=True)

iso_codes = list(biomethane_ammonia_potentials.columns[1:])
countries_names = results_analysis.countries_iso_match(iso_codes)
biomethane_ammonia_potentials = biomethane_ammonia_potentials.T.rename(index=countries_names).reset_index().rename(columns={'index': 'name'})
biomethane_ammonia_potentials['Sustainable biomethane potential in 2030'] = pd.to_numeric(biomethane_ammonia_potentials['Sustainable biomethane potential in 2030'])
biomethane_ammonia_potentials.rename(columns = {'Sustainable biomethane potential in 2030': 'Biomethane potential'}, inplace = True)

# Biomethane-based ammonia potential (Mt/year)
NG_CONSUMPTION_AMMONIA = 0.898843401
biomethane_ammonia_potentials['Ammonia potential'] = biomethane_ammonia_potentials['Biomethane potential'] * NG_CONSUMPTION_AMMONIA

# Ammonia production (Mt/year)
biomethane_ammonia_potentials = biomethane_ammonia_potentials.merge(ammonia_production.rename(columns={'Country': 'name'}), on='name', how='left')
biomethane_ammonia_potentials['Ammonia production'] = biomethane_ammonia_potentials['Ammonia production'].divide(1e9)

# Biomethane required for ammonia production (bcm/year) and share of biomethane (% potential)
biomethane_ammonia_potentials['Biomethane for ammonia production'] = biomethane_ammonia_potentials['Ammonia production'] * NG_CONSUMPTION_AMMONIA
biomethane_ammonia_potentials['Share biomethane for ammonia production'] = biomethane_ammonia_potentials['Biomethane for ammonia production'] * 100 / biomethane_ammonia_potentials['Biomethane potential']

# Average carbon footprint of the EU ammonia industry
carbon_footprint_ammonia_industry = carbon_footprint_ammonia_RER_df.loc['Total'] * biomethane_ammonia_potentials[biomethane_ammonia_potentials['name'] == 'Total']['Ammonia production'].values[0]

In [9]:
# Save results to csv
#biomethane_ammonia_potentials.to_csv(DATA_DIR / "results" / f"Table S2 Sustainable biomethane potential Europe_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index=False)
#carbon_footprint_ammonia_industry.to_csv(DATA_DIR / "results" / f"Table S3 Carbon footprint ammonia industry Europe_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv")

## Blending strategy for net-zero ammonia

In [9]:
# Compute the carbon footprint of ammonia production based on
# the natural gas-biomethane blending strategy as a function of blending ratios
carbon_footprint_blend_range = {}

for scenario in biomethane_fossil_match:
    # European average
    cf_biomethane = carbon_footprint_ammonia_RER_df.loc['Total'][scenario]
    cf_fossil = carbon_footprint_ammonia_RER_df.loc['Total'][biomethane_fossil_match[scenario]]

    carbon_footprint_blend_range[scenario] = {}
    carbon_footprint_blend_range[scenario]['Average'] = results_analysis.carbon_footprint_blending(cf_biomethane, cf_fossil)

    # Country-specific
    for country in LIST_COUNTRIES:
        country_name = results_analysis.countries_iso_match(LIST_COUNTRIES)[country]
        cf_biomethane = carbon_footprint_ammonia_country_df.loc[country][scenario]
        cf_fossil = carbon_footprint_ammonia_country_df.loc[country][biomethane_fossil_match[scenario]]
        
        carbon_footprint_blend_range[scenario][country_name] = results_analysis.carbon_footprint_blending(cf_biomethane, cf_fossil)

In [10]:
# Compute the share of biomethane needed in the blend to achieve net-zero emissions
net_zero_biomethane_ratio = {}
for scenario in carbon_footprint_blend_range:
    net_zero_biomethane_ratio[scenario] = {}
    for location in carbon_footprint_blend_range[scenario]:
        net_zero_biomethane_ratio[scenario][location] = results_analysis.interpolate(carbon_footprint_blend_range[scenario][location])

In [11]:
net_zero_biomethane_ratio_df = pd.DataFrame(net_zero_biomethane_ratio)
net_zero_biomethane_ratio_df.rename(index = {'Average': 'Europe'}, inplace = True)

In [ ]:
# Save results to csv
#net_zero_biomethane_ratio_df.to_csv(DATA_DIR / "results" / f"Fig 4 Net-zero blending ratios_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv")

## Life cycle impact assessment of net-zero ammonia production

Environmental impacts comparison of producing net-zero ammonia using natural gas with CCS, green H2, and biomethane with CCS

In [12]:
# Calculate the impacts for ammonia production from natural gas, green H2, and biomethane
target_inventories = ['Natural gas + CCS Syngas + CCS Heating',
                      'Green H2',
                      'Biomethane + CCS Upgrading + CCS Syngas + CCS Heating']

ammonia_RER_LCIA = {}
for inv in target_inventories:
    activity = [a for a in bw.Database(LCI_DB) if a['name'] == INVENTORIES[inv][0] 
                                               and a['reference product'] == INVENTORIES[inv][1]
                                               and a['location'] == 'RER'][0]
    lca_results = results_analysis.multi_lcia(activity, LCIA_METHODS, amount=1)
    ammonia_RER_LCIA[inv] = lca_results

In [13]:
# Calculate the impacts of DACCS
dac_daccs = [a for a in bw.Database(LCI_DB) if a['name'] == 'carbon dioxide capture, from atmosphere, solid sorbents with heat pumps, with transport and storage, 200 km pipeline storage 1000m'][0]
daccs_LCIA = results_analysis.multi_lcia(dac_daccs, LCIA_METHODS, amount=1)

In [14]:
# Calculate the amount of DAC required in the "Natural gas + CCS Syngas + CCS Heating" and "Green H2" scenarios
daccs_amount = {}
for scenario in ['Natural gas + CCS Syngas + CCS Heating',
                 'Green H2']:
    cf_scenario = ammonia_RER_LCIA[scenario]['Climate change, GWP 100a']

    co2_removal_range = []
    for co2_removal in np.arange(0, 2, 0.05):
      cf_system = cf_scenario + daccs_LCIA['Climate change, GWP 100a'] * co2_removal
      co2_removal_range.append((co2_removal, cf_system))

    co2_removal_range_df = pd.DataFrame(co2_removal_range, columns=['amount', 'impact'])
    s = co2_removal_range_df.set_index('impact').squeeze()
    s.loc[0] = np.nan
    s = s.sort_index().interpolate(method='index')
    daccs_amount[scenario] =s.loc[0]

In [15]:
daccs_amount

{'Natural gas + CCS Syngas + CCS Heating': 1.6856025955052838,
 'Green H2': 0.5122019457767415}

In [20]:
# Calculate the impacts of net zero-ammonia production in the "Natural gas + CCS Syngas + CCS Heating" scenario
ammonia_netzero_RER_LCIA = {}
ammonia_netzero_RER_LCIA.update({'Natural gas + CCS Syngas + CCS Heating': {}})

for impact in LCIA_METHODS:
    if 'Climate change' in impact:
        pass
    else:
        impact_ammonia = ammonia_RER_LCIA['Natural gas + CCS Syngas + CCS Heating'][impact]
        impact_daccs = daccs_amount['Natural gas + CCS Syngas + CCS Heating'] * daccs_LCIA[impact]
            
        ammonia_netzero_RER_LCIA['Natural gas + CCS Syngas + CCS Heating'][impact] = {}
        ammonia_netzero_RER_LCIA['Natural gas + CCS Syngas + CCS Heating'][impact].update({'Total': impact_ammonia + impact_daccs,
                                                                                           'Ammonia production from natural gas with CCS': impact_ammonia,
                                                                                           'DACCS': impact_daccs})

In [21]:
# Calculate the impacts of net zero-ammonia production in the "Green H2" scenario
ammonia_netzero_RER_LCIA.update({'Green H2': {}})

for impact in LCIA_METHODS:
    if 'Climate change' in impact:
        pass
    else:
        impact_ammonia = ammonia_RER_LCIA['Green H2'][impact]
        impact_daccs = daccs_amount['Green H2'] * daccs_LCIA[impact]
            
        ammonia_netzero_RER_LCIA['Green H2'][impact] = {}
        ammonia_netzero_RER_LCIA['Green H2'][impact].update({'Total': impact_ammonia + impact_daccs,
                                                             'Ammonia production from green H2': impact_ammonia,
                                                              'DACCS': impact_daccs})

In [22]:
# Calculate the impacts of net-zero ammonia in the blending strategy scenario
ammonia_netzero_RER_LCIA.update({'Blending strategy': {}})

for impact in LCIA_METHODS:
    blend_ratio = net_zero_biomethane_ratio_df.loc['Europe']["Biomethane + CCS Upgrading + CCS Syngas + CCS Heating"] / 100
    if 'Climate change' in impact:
        pass
    else:
        impact_ng = (1-blend_ratio) * ammonia_RER_LCIA['Natural gas + CCS Syngas + CCS Heating'][impact]
        impact_biomethane = blend_ratio * ammonia_RER_LCIA['Biomethane + CCS Upgrading + CCS Syngas + CCS Heating'][impact]
             
        ammonia_netzero_RER_LCIA['Blending strategy'][impact] = {}
        ammonia_netzero_RER_LCIA['Blending strategy'][impact].update({'Total': impact_ng + impact_biomethane,
                                                                      'Ammonia production from natural gas with CCS': impact_ng,
                                                                      'Ammonia production from biomethane with CCS': impact_biomethane})

In [23]:
# Function to convert the dictionary to a tidy dataframe:
def dict_to_tidy_df(dictionary):
    return pd.DataFrame.from_dict({(i,j): dictionary[i][j] for i in dictionary.keys() 
                                                           for j in dictionary[i].keys()},
                                                           orient='index').reset_index().rename(columns = {'level_0': 'Scenario', 'level_1': 'Category'})

In [24]:
ammonia_netzero_RER_LCIA_df = dict_to_tidy_df(ammonia_netzero_RER_LCIA)

# Calculate relative impacts:
max_impacts = ammonia_netzero_RER_LCIA_df.groupby('Category')[['Total']].max()

ammonia_netzero_RER_LCIA_relative = {}

for scenario in ammonia_netzero_RER_LCIA:
    ammonia_netzero_RER_LCIA_relative[scenario] = {}
    for impact in ammonia_netzero_RER_LCIA[scenario]:
        ammonia_netzero_RER_LCIA_relative[scenario][impact] = {}
        for cat in ammonia_netzero_RER_LCIA[scenario][impact]:
            val = ammonia_netzero_RER_LCIA[scenario][impact][cat]
            val_max = max_impacts.to_dict()['Total'][impact]
            try:
                ammonia_netzero_RER_LCIA_relative[scenario][impact][cat] = val / val_max
            except ZeroDivisionError:
                ammonia_netzero_RER_LCIA_relative[scenario][impact][cat] = 0

ammonia_netzero_RER_LCIA_relative_df = dict_to_tidy_df(ammonia_netzero_RER_LCIA_relative)

In [25]:
# Save results to csv
ammonia_netzero_RER_LCIA_df.to_csv(DATA_DIR / "results" / f"Fig 5 LCIA net zero ammonia scenarios absolute_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index=False)
ammonia_netzero_RER_LCIA_relative_df.to_csv(DATA_DIR / "results" / f"Fig 5 LCIA net zero ammonia scenarios relative_{datetime.datetime.today().strftime('%d-%m-%Y')}.csv", index=False)

## Sensitivity analyses / with presamples

**IMPORTANT**:

In order to avoid the decoding presample json file fails, you need to install the latest presample version, i.e. 0.2.8.

However, the package has not been updated on pypi. To have version 0.2.8, you must install from source:

*python -m pip install git+https://github.com/PascalLesage/presamples.git@master*

Further information here: https://github.com/PascalLesage/presamples/issues/72

**Calculate SA scenarios impacts with presamples**

In [ ]:
scenario_file = Path(DATA_DIR / "sensitivity_analysis_with_presamples.xlsx")

ds_for_sa = [a for a in bw.Database(LCI_DB) if a['name'] == INVENTORIES['Biomethane + CCS Upgrading + CCS Syngas + CCS Heating'][0] 
                                               and a['reference product'] == INVENTORIES['Biomethane + CCS Upgrading + CCS Syngas + CCS Heating'][1]
                                               and a['location'] == 'RER'][0]

SA_lca_results = results_analysis.calculate_impacts_with_presamples(scenario_file, [EI_DB, LCI_DB], ds_for_sa, LCIA_METHODS)


 ps_id, filepath: 675b4db8321b40b9861154755a682b01 C:\Users\istrateir\AppData\Local\pylca\Brightway3\iri_ei39.c70205095bde44795f389c6cb04ff68c\presamples\675b4db8321b40b9861154755a682b01


In [ ]:
SA_SCENARIOS = list(SA_lca_results.columns)
SA_SCENARIOS

['default',
 'SA_Upgrading_WaterScrubbing',
 'SA_Upgrading_ChemicalScrubbing',
 'SA_Upgrading_Membrane',
 'SA_Upgrading_PSA']

In [ ]:
# Calculate the blending ratio for net-zero ammonia for the SA scenarios

# Compute the carbon footprint of ammonia production based on
# the natural gas-biomethane blending strategy as a function of blending ratios
SA_blend_GWP100_range = {}

for scenario in SA_SCENARIOS:
    cf_fossil = ammonia_RER_GWP100.loc['Total'][biomethane_fossil_match['Biomethane + CCS Upgrading + CCS Syngas + CCS Heating']]
    cf_biomethane = SA_lca_results.loc['Climate change, GWP 100a'][scenario]
    SA_blend_GWP100_range[scenario] = carbon_footprint_blending_ratios(cf_biomethane, cf_fossil)
        
# Compute the share of biomethane needed in the blend to achieve net-zero emissions
SA_net_zero_biomethane_ratio = {}
for scenario in SA_blend_GWP100_range:
    SA_net_zero_biomethane_ratio[scenario] = biomethane_share_for_netzero(SA_blend_GWP100_range[scenario])

SA_net_zero_biomethane_ratio_df = pd.DataFrame(SA_net_zero_biomethane_ratio, index=['value']).T

In [ ]:
# Calculate the impacts of the blending strategy
SA_netzero_lcia = {}

for impact in LCIA_METHODS:
    SA_netzero_lcia[impact] = {}
    for scenario in SA_SCENARIOS:
        blend_ratio = SA_net_zero_biomethane_ratio[scenario] / 100
        blend_impact = (1-blend_ratio) * ammonia_RER_LCIA['Natural gas + CCS Syngas + CCS Heating'][impact] + \
                        blend_ratio * SA_lca_results.loc[impact][scenario]
        SA_netzero_lcia[impact][scenario] = blend_impact

SA_netzero_lcia_df = pd.DataFrame(SA_netzero_lcia).T

# Append impacts for the Net-zero blue and Net-zero green scenarios:
netzero_blue_impacts = ammonia_netzero_RER_LCIA_df[(ammonia_netzero_RER_LCIA_df['Scenario'] == "Natural gas + CCS Syngas + CCS Heating")][['Category', 'Total']].set_index('Category')
netzero_green_impacts = ammonia_netzero_RER_LCIA_df[(ammonia_netzero_RER_LCIA_df['Scenario'] == "Green H2")][['Category', 'Total']].set_index('Category')

SA_netzero_lcia_all_df = SA_netzero_lcia_df.iloc[2:].merge(netzero_blue_impacts, left_index=True, right_index=True).merge(netzero_green_impacts, left_index=True, right_index=True)
SA_netzero_lcia_all_df.rename(columns={'Total_x': 'Net-zero blue',
                                       'Total_y': 'Net-zero green',
                                       'default': 'Net-zero biomethane'}, inplace=True)

# Compute the relative impacts:
SA_netzero_lcia_relative_df = SA_netzero_lcia_all_df.div(SA_netzero_lcia_all_df.max(axis=1), axis=0)

In [ ]:
# Save results to csv
SA_net_zero_biomethane_ratio_df.to_csv(DATA_DIR / "results" / "SA_netzero_biomethane_blend.csv")
SA_netzero_lcia_all_df.to_csv(DATA_DIR / "results" / "SA_netzero_ammonia_LCIA_totals.csv")
SA_netzero_lcia_relative_df.to_csv(DATA_DIR / "results" / "SA_netzero_ammonia_LCIA_relative.csv")